# CNN for AdvSND energy reconstruction

In [1]:
from tensorflow.keras.layers import (
    BatchNormalization,
    Concatenate,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    Input,
    MaxPooling2D,
    ReLU,
)
from tensorflow.keras.models import Model

2025-02-11 20:47:27.561414: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-11 20:47:30.667599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from config import input_shape, input_shape_mf

In [3]:
import tensorflow.keras.backend as K

K.set_image_data_format("channels_last")

In [4]:
model_name = "CNN_lyskamm_energy_electron"

In [5]:
drop_middle = 0.25

lr = 2e-4

# AdvTarget

target_h_input = Input(input_shape, name="target_h_in")
X_h = Conv2D(16, kernel_size=(1, 9), padding="same")(target_h_input)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_h)
X_h = Dropout(rate=drop_middle)(X_h)

X_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_h)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_h)
X_h = Dropout(rate=drop_middle)(X_h)

X_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_h)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_h)
X_h = Dropout(rate=drop_middle)(X_h)

X_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_h)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 2), padding="same")(X_h)

X_h = Flatten()(X_h)

target_v_input = Input(input_shape, name="target_v_in")
X_v = Conv2D(16, kernel_size=(1, 9), padding="same")(target_v_input)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_v)
X_v = Dropout(rate=drop_middle)(X_v)

X_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_v)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_v)
X_v = Dropout(rate=drop_middle)(X_v)

X_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_v)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_v)
X_v = Dropout(rate=drop_middle)(X_v)

X_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_v)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 2), padding="same")(X_v)

X_v = Flatten()(X_v)

# AdvTarget

mufilter_h_input = Input(input_shape_mf, name="mufilter_h_in")
X_mf_h = Conv2D(16, kernel_size=(1, 3), padding="same")(mufilter_h_input)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(1, 4), padding="valid")(X_mf_h)
X_mf_h = Dropout(rate=drop_middle)(X_mf_h)

X_mf_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_h)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_mf_h)
X_mf_h = Dropout(rate=drop_middle)(X_mf_h)

X_mf_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_h)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_mf_h)
X_mf_h = Dropout(rate=drop_middle)(X_mf_h)

X_mf_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_h)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(2, 2), padding="same")(X_mf_h)

X_mf_h = Flatten()(X_mf_h)

X = Concatenate()([X_h, X_v, X_mf_h])
X = Dense(3)(X)
X = BatchNormalization()(X)
X = ReLU()(X)
X = Dense(20)(X)
X = BatchNormalization()(X)
X = ReLU()(X)
X = Dropout(rate=0.2)(X)
X = Dense(1)(X)

model = Model(
    inputs=[target_h_input, target_v_input, mufilter_h_input],
    outputs=X,
    name=model_name,
)

K.clear_session()

model.compile(
    optimizer="Adam",
    loss="mse",
    metrics=[
        "mae",
    ],
)

2025-02-11 20:47:34.654397: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-11 20:47:34.955786: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-11 20:47:34.956090: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
model.save(f"{model_name}_e0.keras")